# Extracción de Características con modelo VGGish
## Adrián Arnaiz

# 1. Introducción
Antes de comenzar, cabe destacar que, **con el modelo VGGish solo podemos sacar características para los audios de más de 0.975 segundos.** Esto está explicado en aateriores notebooks y se debe al funcionamiento de VGGish. Para VGGish un ejemplo, una instancia es un frame de 0.96s, sacando ccas para frames de 25ms con window de 10ms (por eso el limite de 0.975). Nosotros para convertir esa matriz de ccas en un vector, hemos hecho la media  desviación por columnas, es decir, de cada característica:

Es decir, **todas las palabras y muchas vocales no podrán ser procesadas por este modelo debido a su corta duración** (el 99% de las palabras duran menos de un segundo y algunos audios de las vocales también).

Por tanto, **realizaremos la extracción de características para read-text y para las vocales que se pueda.**

**Comentar que vamos a sacar de cada audio que podamos, un grupo de ccas que son los embbedings de VGGish y otro que son los MFCC y los espectros. Este grupo ultimo es el paso intermedio de VGGish, pero nosotros las aprovechamos también como posibles características.** Estos ultimos son similares a los que deberíamos haber obtenido con Disvoice. Sin embargo, como no hemos obtenido los resultados esperados y creemos que en parte es por la herramienta Disvoice, vamos a sacar estas características para crear también modelos con ellas.
* Embeddings
* MFCC y espectros


## 1.1 Identificamos los audios 'cortos' (<0.975)
Identificaremos la cantidad de audios menores de esa longitud, para ver si nos quedan suficientes audios en el dataset para crear un clasificador.

Realizaremos comprobación para todas las vocales y todas las palabras. (Todos los read-text son más largos que nuestro límite).

In [1]:
import wave
import contextlib
import os
def identificador_audios_cortos(rutaAud, subaudios, dur):
    d=dict()
    for v in subaudios:
        conthc=0
        contpd=0
        rutaAudios = rutaAud+v+'/'
        rutaAudiosTipo = rutaAudios+'hc/'
        audios = os.listdir(rutaAudiosTipo)      
        for a in audios:
            rut_audio = rutaAudiosTipo+a
            with contextlib.closing(wave.open(rut_audio,'r')) as f:
                frames = f.getnframes()
                rate = f.getframerate()
                duration = frames / float(rate)
                if duration < dur:
                    conthc+=1

        rutaAudios = rutaAud+v+'/'
        rutaAudiosTipo = rutaAudios+'pd/'
        audios = os.listdir(rutaAudiosTipo)      
        for a in audios:
            rut_audio = rutaAudiosTipo+a
            with contextlib.closing(wave.open(rut_audio,'r')) as f:
                frames = f.getnframes()
                rate = f.getframerate()
                duration = frames / float(rate)
                if duration < dur:
                    contpd+=1
                    
        d[v]={'Total':conthc+contpd, 'HC':conthc, 'PD':contpd }
        
    return d

### Vocales

In [2]:
d_voc = identificador_audios_cortos('../PC-GITA/vowels/', ['A','E','I','O','U'], 0.975 )
d_voc

{'A': {'Total': 18, 'HC': 9, 'PD': 9},
 'E': {'Total': 16, 'HC': 7, 'PD': 9},
 'I': {'Total': 19, 'HC': 8, 'PD': 11},
 'O': {'Total': 38, 'HC': 17, 'PD': 21},
 'U': {'Total': 46, 'HC': 26, 'PD': 20}}

De las vocales:
> * A: 18/300 cortos. 282 restantes para clasificación.
* E: 16/300 cortos. 284 restantes para clasificación.
* I : 19/300 cortos. 281 restantes para clasificación.
* O: 38/300 cortos. 262 restantes para clasificación.
* U: 46/300 cortos. 254 restantes para clasificación. 

**Podemos crear clasificadores, nos queda un porcentaje de audios bastante alto respecto al original**

### Palabras

In [3]:
d_pal = identificador_audios_cortos('../PC-GITA/words/', ['atleta','braso','campana','gato','petaka'], 0.975 )
d_pal

{'atleta': {'Total': 95, 'HC': 48, 'PD': 47},
 'braso': {'Total': 100, 'HC': 50, 'PD': 50},
 'campana': {'Total': 99, 'HC': 49, 'PD': 50},
 'gato': {'Total': 100, 'HC': 50, 'PD': 50},
 'petaka': {'Total': 98, 'HC': 50, 'PD': 48}}

De las palabras:
> * atleta: 95/100 cortos. 5 restantes para clasificación.
* braso: 100/100 cortos. 0 restantes para clasificación.
* campana: 99/100 cortos. 1 restantes para clasificación.
* gato: 100/100 cortos. 0 restantes para clasificación.
* petaka: 98/100 cortos. 2 restantes para clasificación. 

**No podemos crear clasificadores, no nos quedan suficientes audios que cumplan las condiciones para VGGish**

Por tanto conluimos que, **realizaremos la extracción de características para read-text y para las vocales que se pueda.** En el caso de las vocales deberemos obviar los audios menores de 0.975s.

----

# 2. Ejemplo uso clase extractora VGGish
Haremos un ejemplo de como usar la clase Extractor_Caracteristicas_Vggish.

Al constructor se le pasa la ruta donde guardar las características extraídas, y opcionalmente el diccionario con información adicional al igual que hacíamos en disvoice. La ruta de características se la pasamos para que compruebe si el directorio existe y si no es así lo cree. Es decir, no guarda automáticamente ahí las caracetrísticas las cuales se nos devolveran en las funciones y seremos nosotros quien las guardaremos en ese directorio. Lo único que hace es crearlo.

Se ha realizado de este modo debido a que en la mayoría de los casos queremos explorar las características antes de guardarlas.

In [4]:
from extractor_ccas_vggish import Extractor_Caracteristicas_Vggish
import numpy as np
import pandas as pd

Using TensorFlow backend.


Recuperamos el diccionario con información del sexo y edad del paciente.

In [5]:
import pickle
#recuperamos el diccionario de la info de sexo y edad
pickle_in = open("../CaracteristicasExtraidas/dict_audios_inf.pickle","rb")
dic_audios_inf = pickle.load(pickle_in)

Creamos el objeto Extractor

In [6]:
rutaCcas = 'PruebaCcasVggish/'
extractor = Extractor_Caracteristicas_Vggish(rutaCcas,dic_audios_inf)

Instructions for updating:
Colocations handled automatically by placer.


Para extraer características usamos la función extraccion_embeddings_directorio, que sigue el mismo patrón de funcionamiento que la funcion extraccion_ccas_directorio de el extractor de características Disvoice.

Le pasamos la ruta donde se encuentran los audios divididos en dos carpetas hc y pd. También le pasamos opcionalmente los atributos adicionales a añadir. Estos atributos adicionales deberán el nombre que tienen en el diccionario anteriormente importado.

In [7]:
vggish_rt_ccas = extractor.extraccion_embeddings_directorio('PC-GITA/read-text/',['SEX','AGE'])

Comienzo extracción HC, quedan: 100 audios.
Comienzo extracción PD, quedan: 50 audios.


Mostramos las dimensiones: n_audios X (ccas_extraidas+atribs_adicionales+Label)

In [8]:
vggish_rt_ccas.shape

(100, 259)

Vemos como las 3 ultimas columas son Sexo (1-F, 0-M), Edad y PD/HC(1/0)

In [9]:
import pandas as pd
pd.DataFrame(vggish_rt_ccas).head()

,0,1,2,3,4,5,6,7,8,9,...,249,250,251,252,253,254,255,256,257,258
0,0.283398,0.005795,0.322382,0.0,0.007770,0.002156,0.007077,0.000000,0.015446,0.016417,...,0.093643,0.0,0.213005,0.308270,0.065252,0.000000,0.000000,0.0,64.0,0.0
1,0.576316,0.006670,0.337802,0.0,0.003554,0.005937,0.001004,0.000000,0.000000,0.030303,...,0.096462,0.0,0.234707,0.106396,0.169872,0.036177,0.028395,1.0,72.0,0.0
2,0.640286,0.000000,0.263864,0.0,0.000974,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.078060,0.0,0.232578,0.205344,0.046105,0.000000,0.000000,1.0,75.0,0.0
3,0.215650,0.053099,0.061410,0.0,0.022464,0.010151,0.000000,0.022203,0.000000,0.015516,...,0.046254,0.0,0.077390,0.161684,0.145853,0.017135,0.000000,0.0,65.0,0.0
4,0.376299,0.000000,0.299862,0.0,0.001700,0.000000,0.006846,0.000000,0.008140,0.025373,...,0.078498,0.0,0.108306,0.142625,0.000000,0.000000,0.000000,1.0,66.0,0.0


In [10]:
!rmdir ..\PruebaCcasVggish

---

# 3. Extracción Embeddings VGGish
**Como hemos comentado extraeremos las características de VGGish embeddings para los read-text y para cada una de las vocales.**

> Audio -->**|Preprocesado|**--> MFCC y espectros para cada segundo -->**|VGGish|**-->Embeddings-->**|Media y desviacion|**--> final embeddings

### 3.1 Extracción VGGish embeddings read-text

In [11]:
rutaCcas = 'CaracteristicasExtraidas/vggish/embbedings/'
extractor = Extractor_Caracteristicas_Vggish(rutaCcas,dic_audios_inf)

In [12]:
vggish_embed_rt_ccas = extractor.extraccion_embeddings_directorio('PC-GITA/read-text/',['SEX','AGE'])

Comienzo extracción HC, quedan: 100 audios.
Comienzo extracción PD, quedan: 50 audios.


In [13]:
assert vggish_embed_rt_ccas.shape == (100, 259)

In [14]:
pd.DataFrame(vggish_embed_rt_ccas).head()

,0,1,2,3,4,5,6,7,8,9,...,249,250,251,252,253,254,255,256,257,258
0,0.283398,0.005795,0.322382,0.0,0.007770,0.002156,0.007077,0.000000,0.015446,0.016417,...,0.093643,0.0,0.213005,0.308270,0.065252,0.000000,0.000000,0.0,64.0,0.0
1,0.576316,0.006670,0.337802,0.0,0.003554,0.005937,0.001004,0.000000,0.000000,0.030303,...,0.096462,0.0,0.234707,0.106396,0.169872,0.036177,0.028395,1.0,72.0,0.0
2,0.640286,0.000000,0.263864,0.0,0.000974,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.078060,0.0,0.232578,0.205344,0.046105,0.000000,0.000000,1.0,75.0,0.0
3,0.215650,0.053099,0.061410,0.0,0.022464,0.010151,0.000000,0.022203,0.000000,0.015516,...,0.046254,0.0,0.077390,0.161684,0.145853,0.017135,0.000000,0.0,65.0,0.0
4,0.376299,0.000000,0.299862,0.0,0.001700,0.000000,0.006846,0.000000,0.008140,0.025373,...,0.078498,0.0,0.108306,0.142625,0.000000,0.000000,0.000000,1.0,66.0,0.0


In [15]:
np.save('../'+rutaCcas+'vggish_embed_rt_ccas', vggish_embed_rt_ccas)

### 3.2 Extracción VGGish embeddings vocales
En este caso no añadiremos la edad y el sexo debido a la dificultad añadida de no tener todos los audios computados: tenemos que eliminar los de menos de un segundo.

#### A

In [16]:
vggish_embed_v_A_ccas = extractor.extraccion_embeddings_directorio('PC-GITA/vowels/A/')

Comienzo extracción HC, quedan: 300 audios.


C:\Users\usuario\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\usuario\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\usuario\Anaconda3\lib\site-packages\numpy\core\_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Users\usuario\Anaconda3\lib\site-packages\numpy\core\_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\Users\usuario\Anaconda3\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Comienzo extracción PD, quedan: 150 audios.


In [17]:
assert vggish_embed_v_A_ccas.shape == (300-d_voc['A']['Total'],257)
vggish_embed_v_A_ccas.shape

(282, 257)

In [18]:
np.save('../'+rutaCcas+'vggish_embed_v_A_ccas', vggish_embed_v_A_ccas)

#### E

In [19]:
vggish_embed_v_E_ccas = extractor.extraccion_embeddings_directorio('PC-GITA/vowels/E/')

Comienzo extracción HC, quedan: 300 audios.
Comienzo extracción PD, quedan: 150 audios.


In [20]:
assert vggish_embed_v_E_ccas.shape == (300-d_voc['E']['Total'],257)
vggish_embed_v_E_ccas.shape

(284, 257)

In [21]:
np.save('../'+rutaCcas+'vggish_embed_v_E_ccas', vggish_embed_v_E_ccas)

#### I

In [22]:
vggish_embed_v_I_ccas = extractor.extraccion_embeddings_directorio('PC-GITA/vowels/I/')

Comienzo extracción HC, quedan: 300 audios.
Comienzo extracción PD, quedan: 150 audios.


In [23]:
assert vggish_embed_v_I_ccas.shape == (300-d_voc['I']['Total'],257)
vggish_embed_v_I_ccas.shape

(281, 257)

In [24]:
np.save('../'+rutaCcas+'vggish_embed_v_I_ccas', vggish_embed_v_I_ccas)

#### O

In [25]:
vggish_embed_v_O_ccas = extractor.extraccion_embeddings_directorio('PC-GITA/vowels/O/')

Comienzo extracción HC, quedan: 300 audios.
Comienzo extracción PD, quedan: 150 audios.


In [26]:
assert vggish_embed_v_O_ccas.shape == (300-d_voc['O']['Total'],257)
vggish_embed_v_O_ccas.shape

(262, 257)

In [27]:
np.save('../'+rutaCcas+'vggish_embed_v_O_ccas', vggish_embed_v_O_ccas)

#### U

In [28]:
vggish_embed_v_U_ccas = extractor.extraccion_embeddings_directorio('PC-GITA/vowels/U/')

Comienzo extracción HC, quedan: 300 audios.
Comienzo extracción PD, quedan: 150 audios.


In [29]:
assert vggish_embed_v_U_ccas.shape == (300-d_voc['U']['Total'],257)
vggish_embed_v_U_ccas.shape

(254, 257)

In [30]:
np.save('../'+rutaCcas+'vggish_embed_v_U_ccas', vggish_embed_v_U_ccas)

---

# 4. Extracción MFCC y espectros VGGish

Como hemos comentado, también aprovecharemos el anterior paso a el calculo de los embbedings del modelo para sacar las características de espectros y MFCCs.

El prepocesado realiza lo siguiente:
>* Todo el audio se remuestrea a 16 kHz mono.
* Un espectrograma se calcula utilizando magnitudes de la Transformada de Fourier a corto plazo con un tamaño de ventana de 25 ms, un salto de ventana de 10 ms y una ventana de Hann periódica.
* Un espectrograma de mel se calcula al mapear el espectrograma a 64 casillas de mel que cubren el rango de 125-7500 Hz.
* Se calcula un espectrograma log mel estabilizado aplicando log (espectro de mel + 0.01) donde se usa el desplazamiento para evitar tomar un logaritmo de cero.
* **Estas características se enmarcan en ejemplos no superpuestos de 0.96 segundos, donde cada ejemplo cubre 64 bandas mel y 96 frames de 25 ms  con windowing 10ms cada una.**

**Nosotros aplanamos el vector de (X,96,64) a (X\*96,64). Posteriormente hacemos la media y desviación de cada una de las 64 características a lo largo de los X\*96 frames analizados. Y obtenemos para cada audio un vector de 128 ccas (mas adicionales si es necesario y label).**

> Audio -->**|Preprocesado|**--> MFCC y espectros para cada segundo -->**|Aplanados|**-->**|Media y desviacion|**--> final espectros

In [31]:
rutaCcas = 'CaracteristicasExtraidas/vggish/espectros/'
extractor = Extractor_Caracteristicas_Vggish(rutaCcas,dic_audios_inf)

In [32]:
vggish_espec_rt_ccas = extractor.extraccion_embeddings_directorio('PC-GITA/read-text/',['SEX','AGE'],False)

Comienzo extracción HC, quedan: 100 audios.
Comienzo extracción PD, quedan: 50 audios.


In [33]:
assert vggish_espec_rt_ccas.shape == (100, 131)

In [34]:
vggish_espec_rt_ccas.shape

(100, 131)

In [35]:
pd.DataFrame(vggish_espec_rt_ccas).head()

,0,1,2,3,4,5,6,7,8,9,...,121,122,123,124,125,126,127,128,129,130
0,-2.289717,-1.917971,-1.615449,-1.501580,-1.626233,-1.755354,-1.814383,-1.879997,-1.810921,-1.933020,...,0.992340,0.972268,0.923451,0.881331,0.916335,0.925145,0.963934,0.0,64.0,0.0
1,-1.868412,-1.900832,-1.678849,-1.373311,-1.362722,-1.456512,-1.506287,-1.566286,-1.583171,-1.838715,...,0.864394,0.891869,0.930504,0.920924,0.921429,0.861128,0.771751,1.0,72.0,0.0
2,-1.944845,-1.551957,-1.500288,-1.701158,-1.985579,-1.851621,-1.557754,-1.362149,-1.302102,-1.583566,...,0.738100,0.791338,0.789183,0.803143,0.910439,0.955155,0.911863,1.0,75.0,0.0
3,-0.115094,-0.245233,-0.432217,-0.465459,-0.266510,0.156148,0.446809,0.435701,0.210680,-0.343895,...,1.525810,1.463179,1.382741,1.328618,1.220216,1.231141,1.301151,0.0,65.0,0.0
4,-2.421993,-1.901072,-1.640123,-1.615278,-1.956334,-2.221964,-2.152975,-2.022032,-1.796112,-1.847532,...,1.159509,1.217289,1.133309,1.111786,1.113843,1.141836,1.199390,1.0,66.0,0.0


In [36]:
np.save('../'+rutaCcas+'vggish_espec_rt_ccas', vggish_espec_rt_ccas)

### 3.2 Extracción VGGish embeddings vocales
En este caso no añadiremos la edad y el sexo debido a la dificultad añadida de no tener todos los audios computados: tenemos que eliminar los de menos de un segundo.

#### A

In [39]:
vggish_espec_v_A_ccas = extractor.extraccion_embeddings_directorio('PC-GITA/vowels/A/',embeddings=False)

Comienzo extracción HC, quedan: 300 audios.


C:\Users\usuario\Anaconda3\lib\site-packages\numpy\core\_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
C:\Users\usuario\Anaconda3\lib\site-packages\numpy\core\_methods.py:130: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


Comienzo extracción PD, quedan: 150 audios.


In [40]:
assert vggish_espec_v_A_ccas.shape == (300-d_voc['A']['Total'],129)
vggish_espec_v_A_ccas.shape

(282, 129)

In [41]:
np.save('../'+rutaCcas+'vggish_espec_v_A_ccas', vggish_espec_v_A_ccas)

#### E

In [42]:
vggish_espec_v_E_ccas = extractor.extraccion_embeddings_directorio('PC-GITA/vowels/E/',embeddings=False)

Comienzo extracción HC, quedan: 300 audios.
Comienzo extracción PD, quedan: 150 audios.


In [43]:
assert vggish_espec_v_E_ccas.shape == (300-d_voc['E']['Total'],129)
vggish_espec_v_E_ccas.shape

(284, 129)

In [44]:
np.save('../'+rutaCcas+'vggish_espec_v_E_ccas', vggish_espec_v_E_ccas)

#### I

In [45]:
vggish_espec_v_I_ccas = extractor.extraccion_embeddings_directorio('PC-GITA/vowels/I/',embeddings=False)

Comienzo extracción HC, quedan: 300 audios.
Comienzo extracción PD, quedan: 150 audios.


In [46]:
assert vggish_espec_v_I_ccas.shape == (300-d_voc['I']['Total'],129)
vggish_espec_v_I_ccas.shape

(281, 129)

In [47]:
np.save('../'+rutaCcas+'vggish_espec_v_I_ccas', vggish_espec_v_I_ccas)

#### O

In [48]:
vggish_espec_v_O_ccas = extractor.extraccion_embeddings_directorio('PC-GITA/vowels/O/',embeddings=False)

Comienzo extracción HC, quedan: 300 audios.
Comienzo extracción PD, quedan: 150 audios.


In [49]:
assert vggish_espec_v_O_ccas.shape == (300-d_voc['O']['Total'],129)
vggish_espec_v_O_ccas.shape

(262, 129)

In [50]:
np.save('../'+rutaCcas+'vggish_espec_v_O_ccas', vggish_espec_v_O_ccas)

#### U

In [51]:
vggish_espec_v_U_ccas = extractor.extraccion_embeddings_directorio('PC-GITA/vowels/U/',embeddings=False)

Comienzo extracción HC, quedan: 300 audios.
Comienzo extracción PD, quedan: 150 audios.


In [52]:
assert vggish_espec_v_U_ccas.shape == (300-d_voc['U']['Total'],129)
vggish_espec_v_U_ccas.shape

(254, 129)

In [53]:
np.save('../'+rutaCcas+'vggish_espec_v_U_ccas', vggish_espec_v_U_ccas)

---